In [2]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import cv2
import dlib
import time
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Load dataset

In [28]:
data = pd.read_pickle('faces_all.pkl')
detector = dlib.get_frontal_face_detector()
p = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor (p)
face_rec_model_path = 'dlib_face_recognition_resnet_model_v1.dat'
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

In [12]:
def get_faces(file):
    """
        This function has the same functionality as in 2-Get_face_descriptor.ipynb
        It returns a pandas dataframe contains the face decriptors
    """
    faces = []
    image = cv2.imdecode(np.fromfile(file,dtype=np.uint8),-1)
    image = cv2. cvtColor (image, cv2.COLOR_BGR2RGB)
    gray = cv2. cvtColor (image, cv2.COLOR_RGB2GRAY)

    rects = detector (gray , 0)
    if len(rects) >=0:
        for rect in rects :
            shape = predictor (gray , rect )
            face_chip = dlib.get_face_chip(image, shape)
            face_descriptor = facerec.compute_face_descriptor(face_chip)
            item = [file, [rect.left(), rect.top(), rect.width(), rect.height()], list(face_descriptor)]
            faces.append(item)
    return pd.DataFrame(faces, columns=['file', 'rect', 'face_descriptor'])

In [4]:
def rec_person(row):
    """
        This function takes a file's path (better absolute path) as argument and returns the image's information with 
        the most likely match
    """
    max_d = 1000
    person = None
    for i in range(len(data)):
        f = data.iloc[i]
        #print(f)
        if f.file == row.file:
            continue
        #print(row.face_descriptor)
        d = np.linalg.norm(np.array(f.face_descriptor) - np.array(row.face_descriptor))
        if d < max_d:
            max_d = d
            person = f
    if max_d<0.6:
        return([max_d,person])
    else:
        return None

In [18]:
def rec_persons(row):
    """
        This function takes a file's path (better absolute path) as argument and returns all images with similar face.
        The result is a list of pandas series
    """
    persons = []
    for i in range(len(data)):
        f = data.iloc[i]
        #print(f)
        if f.file == row.file:
            continue
        #print(row.face_descriptor)
        d = np.linalg.norm(np.array(f.face_descriptor) - np.array(row.face_descriptor))
        if d < 0.3:
            persons.append(f)
    return persons

In [13]:
img = "D:\\Data2\\new\\8.29张江人工智能岛\\29号\\摄影勇\\DCIM\\101EOS5D\\7A4A7831.JPG" # Path to sample image for searching
img_info = get_faces(img)
img_info

In [29]:
for i in range(len(img_info)):
    print("For image: ", img_info.iloc[i]['file'],":")
    result = rec_persons(img_info.iloc[i])
    for j in range(len(result)):
        print(result[j]['file'])
    print()

For image:  D:\Data2\new\8.29张江人工智能岛\29号\摄影勇\DCIM\101EOS5D\7A4A7831.JPG :
D://Data2\new\8.29张江人工智能岛\29号\摄影勇\DCIM\100EOS5D\7A4A7393.JPG
D://Data2\new\8.29张江人工智能岛\29号\摄影勇\DCIM\101EOS5D\7A4A7831.JPG
D://Data2\new\8.29张江人工智能岛\29号\摄影勇\DCIM\101EOS5D\7A4A7832.JPG
D://Data2\new\8.29张江人工智能岛\29号\摄影勇\DCIM\102EOS5D\7A4A8019.JPG

For image:  D:\Data2\new\8.29张江人工智能岛\29号\摄影勇\DCIM\101EOS5D\7A4A7831.JPG :
D://Data2\new\8.29张江人工智能岛\29号\摄影勇\DCIM\100EOS5D\7A4A7387.JPG
D://Data2\new\8.29张江人工智能岛\29号\摄影勇\DCIM\101EOS5D\7A4A7831.JPG
D://Data2\new\8.29张江人工智能岛\29号\摄影勇\DCIM\101EOS5D\7A4A7832.JPG
D://Data2\new\8.29张江人工智能岛\29号\摄影勇\DCIM\101EOS5D\7A4A7841.JPG
D://Data2\new\8.29张江人工智能岛\29号\摄影勇\DCIM\101EOS5D\7A4A7988.JPG
D://Data2\new\8.29张江人工智能岛\31号\照片\摄影 迟\0P0A1197.JPG
D://Data2\new\8.29张江人工智能岛\31号\照片\摄影 迟\0P0A1226.JPG
D://Data2\new\8.29张江人工智能岛\31号\照片\摄影 迟\0P0A1230.JPG
D://Data2\new\8.29张江人工智能岛\31号\照片\摄影 迟\0P0A1233.JPG
D://Data2\new\8.29张江人工智能岛\31号\照片\摄影 迟\0P0A1235.JPG
D://Data2\new\8.29张江人工智能岛\31号\照片\摄影 迟\0P0A1236.